In [26]:
最適化やり方
関数：(モデルを生成→各種回収率を算出)x10 max(その平均-1.96*SE)を出力
パラメータ：max_depth,num_leaves

In [55]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import graphviz
options = ChromeOptions()
options.add_argument('--headless')   
import warnings
warnings.simplefilter('ignore')

def training(dfs,odds_sim=False):
    for df in dfs:
        df.reset_index(inplace=True, drop=True)
        df = df.drop(columns=["Unnamed: 0"])
        df["studium"] = [str(df.loc[i,"レース番号"])[5] for i in range(len(df))]
        df["round"] = [str(df.loc[i,"レース番号"])[-2:] for i in range(len(df))]
        df["year"] = [str(df.loc[i,"レース番号"])[:4] for i in range(len(df))]
        df["着順"] = [18-df["着順"][i] for i in range(len(df))]
        #df["着順"] = list(map(lambda x: x if x >=13 else 0 ,df["着順"]))

    le = {}
    for i in ["馬名","騎手","芝/ダ","天気","馬場","レース番号","studium","round","year"]:
        le[i] = preprocessing.LabelEncoder()
        le[i].fit(np.concatenate(dfs[0][i].values,dfs[1][i].values))
        dfs[0][i] = le[i].transform(dfs[0][i])
        dfs[1][i] = le[i].transform(dfs[1][i])
    
    for df in dfs:
        df = df.rename(columns={
            '馬名':'horse_name',
            '馬番':'horse_num',
            '着順':'rank',
            '馬齢':'age',
            "騎手":"jockey",
            "単勝オッズ":"odds",
            "レース番号":"race_num",
            "芝/ダ":"g_d",
            "距離":"length",
            "天気":"weather",
            "馬場":"baba",
            "賞金":"prize"
        })
    
    id_list = [i for i in range(dfs[1].loc[len(dfs[1])-1,"race_num"])]
    
    garbage, id_test_test = train_test_split(id_test,test_size=0.3,shuffle=True)
    df_train,df_test,df_test_test = dfs[0],dfs[1],dfs[1].query("race_num in @id_test_test")
 
    q_train = df_train.groupby('race_num').size().values
    q_test = df_test.groupby('race_num').size().values
    df_train.drop(columns="race_num",inplace=True)
    df_test.drop(columns="race_num",inplace=True)
    y_train,y_test = df_train["rank"],df_test["rank"]
    odds_train,odds_test = df_train["odds_all"],df_test["odds_all"]
    X_train,X_test = df_train.drop(columns=["odds","rank","odds_all"]),df_test.drop(columns=["odds","rank","odds_all"])
    #X_train["odds"] = odds_train
    #X_test["odds"] = odds_test
  
    train = lgb.Dataset(X_train, y_train,group=q_train)
    test = lgb.Dataset(X_test, y_test, group=q_test)
    

    params = { 
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'ndcg_eval_at': 3,
        'max_depth': 5,
        'max_position':eval_at, #最適化するndcg@k
        'num_leaves': 32,
        'min_data_in_leaf':1,
        'verbose': -1,
        'categorical_feature':[0,1,3,4,6,7,9,10,11],
        'boosting_type': 'gbdt'
        
    }

    model = lgb.train(
        params,
        train,
        num_boost_round=150,
        valid_sets=[train,test],
        valid_names=['train','test'],
        early_stopping_rounds=30,
        verbose_eval=False
    )
    
    if odds_sim == False:
        return model,le
    
    else:
        #3番目まで全てかけてみた
        from_ = 0
        odds_test = df_test_test["odds_all"]
        odds_all = odds_test.values
        hit = [0,0,0,0,0,0,0]
        get = np.array([0,0,0,0,0,0,0])
        q_test = df_test_test.groupby('race_num').size().values
        df_test_test.drop(columns="race_num",inplace=True)
        y_test = df_test_test["rank"]
        
        for i in q_test:
            pred = model.predict(X_test.iloc[from_:from_+i,:],num_iteration=model.best_iteration)
            pred = pd.Series(pred).sort_values(ascending=False)          
            pred.index = pred.index + 1
            pred_rank = list(map(str,pred.index.values))[0:3]
            
            odds = odds_all[from_].replace(",","")
            from_ += i
            #単勝|6|190円;複勝|6 5 9|130円430円350円;枠連|3 3|2,000円;馬連|5 6|1,870円;
            #ワイド|5 6 6 9 5 9|720円610円3,010円;馬単|6 5|2,560円;3連複|5 6 9|7,200円;3連単|6 5 9|25,490円
            odds_list = re.findall(r'([^\|]*)\|([^\|]*)\|([^;]*)円;?',odds)
        
            result = []
            for i,j in zip([1,2],[" ","円"]):
                comb_list =  [x[i] for x in odds_list if x[0] != "枠連"]
                comb_list[0] = [comb_list[0:1]]
                comb_list[1] = [[a] for a in comb_list[1].split(j)]
                if len(comb_list[1]) == 5:
                    comb_list[1] = comb_list[1][::2]
                    j = "   "
                comb_list[2] = [set(comb_list[2].split(j))]
                comb_list[3] = comb_list[3].split(j)
                comb_list[3] = [set(comb_list[3][0:(3-i)]),set(comb_list[3][3-i:6-2*i]),set(comb_list[3][6-2*i:9-3*i])]
                comb_list[4] = [comb_list[4].split(j)]
                comb_list[5] = [set(comb_list[5].split(j))]
                comb_list[6] = [comb_list[6].split(j)]
                result.append(comb_list)
            
            baken_list = [0,0,0,0,0,0,0]
            baken_list[0] = [[a] for a in pred_rank[0:3]]
            baken_list[1] = [[a] for a in pred_rank[0:3]]
            baken_list[2] = [set(pred_rank[0:2]),set(pred_rank[1:3]),set(pred_rank[::2])]
            baken_list[3] = [set(pred_rank[0:2]),set(pred_rank[1:3]),set(pred_rank[::2])]
            baken_list[4] = [pred_rank[0:2]]
            baken_list[5] = [set(pred_rank)]
            baken_list[6] = [pred_rank]
         
            for i in range(len(result[0])):
                for j in range(len(result[0][i])):
                    if result[0][i][j] in baken_list[i]:
                        hit[i] += 1
                        #print(result[0][i],result[1][i])
                        get[i] += int(list(result[1][i][j])[0])
            
           
            
        for l in [hit,get]:
            l[0] = l[0]/3
            l[1] = l[1]/3
            l[2] = l[2]/3
            l[3] = l[3]/3
            l[4] = l[4]
            l[6] = l[6]
        
        get = get/len(q_test)
        return get
        """
        print_list = ["単勝:","複勝:","馬連:","ワイド:","馬単:","3連複:","3連単:"]

        for i in range(len(print_list)):
            print(print_list[i]+"的中率→{0}%,回収率→{1}%".format(hit[i]*100/len(q_test),get[i]/len(q_test)))
        
        #graph = lgb.create_tree_digraph(model, tree_index=0, name='Tree')
        #graph.render(view=True)
        #print(len(q_test))
        #importance = pd.DataFrame(model.feature_importance(), index=X_train.columns.values, columns=['importance'])
        return importance
        """

    
    
    



In [56]:
df = pd.DataFrame()
for k in [[2015,2016]]:#,[2016,2017],[2017,2018],[2018,2019],[2019,2020]]:
    dfs = []
    for i in k:
        dfs.append(pd.read_csv("horses_data_"+str(i)+".csv"))
        
    get_store=[[],[],[],[],[],[],[]]
    for i in range(10):
        get = training(dfs,1)
        for j in range(len(get)):
            get_store[j].append(get[j])

    for j in get_store:
        se = np.std(j,ddof=1)/np.sqrt(10)
        ave = np.average(j)
    print(ave-1.96*se,ave+1.96*se)

    
    

TypeError: only integer scalar arrays can be converted to a scalar index

In [37]:
for j in get_store:
    print(np.average(j),np.std(j,ddof=1)/np.sqrt(10))

79.65929029078363 0.6507879849305017
82.36944307540661 0.5016847551807011
79.18398225726959 2.7527420419675055
81.03765401675703 1.3232283776514715
77.61015278462295 3.2154447917061697
78.42828979793003 8.030510318020815
62.89551503203548 7.503366962619427


In [5]:
text = "単勝|6|190円;複勝|6 5 9|130円430円350円;枠連|3 3|2,000円;馬連|5 6|1,870円;ワイド|5 6 6 9 5 9|720円610円3,010円;馬単|6 5|2,560円;3連複|5 6 9|7,200円;3連単|6 5 9|25,490円"
text = text.replace(",","")
odds_list = re.findall(r'([^\|]*)\|([^\|]*)\|([^;]*)円;?',text)
result = []
for i,j in zip([1,2],[" ","円"]):
    comb_list =  [x[i] for x in odds_list if x[0] != "枠連"]
    #haito_list = [x[2] for x in odds_list if x[0] != "枠連"]
    comb_list[0] = [comb_list[0:1]]
    comb_list[1] = [[a] for a in comb_list[1].split(j)]
    if len(comb_list[1]) == 5:
        comb_list[1] = comb_list[1][::2]
        j = "   "
    comb_list[2] = [set(comb_list[2].split(j))]
    comb_list[3] = comb_list[3].split(j)
    comb_list[3] = [set(comb_list[3][0:(3-i)]),set(comb_list[3][3-i:6-2*i]),set(comb_list[3][6-2*i:9-3*i])]
    comb_list[4] = [comb_list[4].split(j)]
    comb_list[5] = [set(comb_list[5].split(j))]
    comb_list[6] = [comb_list[6].split(j)]
    result.append(comb_list)

print(result)

pred_rank =['6','4','9']
baken_list = [0,0,0,0,0,0,0]
baken_list[0] = [pred_rank[0:1]]
baken_list[1] = [[a] for a in pred_rank[0:len(result[0][1])]]
baken_list[2] = [set(pred_rank[0:2])]
baken_list[3] = [set(pred_rank[0:2]),set(pred_rank[1:3]),set(pred_rank[::2])]
baken_list[4] = [pred_rank[0:2]]
baken_list[5] = [set(pred_rank)]
baken_list[6] = [pred_rank]
    
print(baken_list)
hit = [0,0,0,0,0,0,0]
get = [0,0,0,0,0,0,0]

for i in range(len(baken_list)):
    for j in range(len(baken_list[i])):
        if result[0][i][j] in baken_list[i]:
            hit[i] += 1
            get[i] += int(list(result[1][i][j])[0])
            print(get)

for l in [hit,get]:
    l[0] = l[0]/3
    l[1] = l[1]/3
    l[2] = l[2]/3
    l[3] = l[3]/3
    l[4] = l[4]/6
    l[6] = l[6]/6

print_list = ["単勝:","複勝:","馬連:","ワイド:","馬単:","3連複:","3連単:"]

for i in range(len(print_list)):
    print(print_list[i]+"的中率→{0}%,回収率→{1}%".format(hit[i]*100,get[i]))
        
        


[[[['6']], [['6'], ['5'], ['9']], [{'5', '6'}], [{'5', '6'}, {'9', '6'}, {'5', '9'}], [['6', '5']], [{'5', '9', '6'}], [['6', '5', '9']]], [[['190']], [['130'], ['430'], ['350']], [{'1870'}], [{'720'}, {'610'}, {'3010'}], [['2560']], [{'7200'}], [['25490']]]]
[[['6']], [['6'], ['4'], ['9']], [{'4', '6'}], [{'4', '6'}, {'9', '4'}, {'9', '6'}], [['6', '4']], [{'4', '9', '6'}], [['6', '4', '9']]]
[190, 0, 0, 0, 0, 0, 0]
[190, 130, 0, 0, 0, 0, 0]
[190, 480, 0, 0, 0, 0, 0]
[190, 480, 0, 610, 0, 0, 0]
単勝:的中率→33.33333333333333%,回収率→63.333333333333336%
複勝:的中率→66.66666666666666%,回収率→160.0%
馬連:的中率→0.0%,回収率→0.0%
ワイド:的中率→33.33333333333333%,回収率→203.33333333333334%
馬単:的中率→0.0%,回収率→0.0%
3連複:的中率→0%,回収率→0%
3連単:的中率→0.0%,回収率→0.0%


In [210]:
del pred_rank

In [158]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
options = ChromeOptions()
options.add_argument('--headless')   
import warnings
warnings.simplefilter('ignore')

def training_umaren(df,odds_sim=False):
    
    df.reset_index(inplace=True, drop=True)
    df = df.drop(columns=["Unnamed: 0"])
    df["studium"] = [str(df.loc[i,"レース番号"])[5] for i in range(len(df))]
    df["round"] = [str(df.loc[i,"レース番号"])[-2:] for i in range(len(df))]
    df["year"] = [str(df.loc[i,"レース番号"])[:4] for i in range(len(df))]
    df["着順"] = [18-df["着順"][i] for i in range(len(df))]
    #df["着順"] = list(map(lambda x: x if x >=13 else 0 ,df["着順"]))
    
    le = {}
    for i in ["馬名","騎手","芝/ダ","天気","馬場","レース番号","studium","round","year"]:
        le[i] = preprocessing.LabelEncoder()
        le[i].fit(df[i])
        df[i] = le[i].transform(df[i])
    df = df.rename(columns={
        '馬名':'horse_name',
        '馬番':'horse_num',
        '着順':'rank',
        '馬齢':'age',
        "騎手":"jockey",
        "単勝オッズ":"odds",
        "レース番号":"race_num",
        "芝/ダ":"g_d",
        "距離":"length",
        "天気":"weather",
        "馬場":"baba",
        "賞金":"prize"
    })
   
    id_list = [i for i in range(df.loc[len(df)-1,"race_num"])]
    
    id_train,id_test = train_test_split(id_list,test_size=0.2,shuffle=False)
    df_train,df_test = df.query("race_num in @id_train"),df.query("race_num in @id_test")
  
   
    q_train = df_train.groupby('race_num').size().values
    q_test = df_test.groupby('race_num').size().values
    df_train.drop(columns="race_num",inplace=True)
    df_test.drop(columns="race_num",inplace=True)
    y_train,y_test = df_train["rank"],df_test["rank"]
    odds_train,odds_test = df_train["odds"],df_test["odds"]
    odds_umaren_test = df_test["odds_umaren"]
    X_train,X_test = df_train.drop(columns=["odds","rank","odds_umaren"]),df_test.drop(columns=["odds","rank","odds_umaren"])
    #X_train["odds"] = odds_train
    #X_test["odds"] = odds_test
    #print(X_train.columns.values[[0,1,3,4,6,7,9,10,11]])

    train = lgb.Dataset(X_train, y_train, categorical_feature=[0,1,3,4,6,7,9,10,11], group=q_train)
    test = lgb.Dataset(X_test, y_test, categorical_feature=[0,1,3,4,6,7,9,10,11], group=q_test)
    
    eval_at = 3
    params = { # eval_at:3 num_leaves:200 min_data_in_leaf :100
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'ndcg_eval_at': [eval_at],
        'max_depth': 20,
        'max_position':eval_at, #最適化するndcg@k
        'num_leaves': 300,
        'min_data_in_leaf':1,
        'boosting_type': 'gbdt'
        
    }

    model = lgb.train(
        params,
        train,
        num_boost_round=150,
        valid_sets=[train,test],
        valid_names=['train','test'],
        early_stopping_rounds=20,
        verbose_eval = 1
    )
    
    if odds_sim == False:
        return model,le
    
    else:
        #3番目までに全てかけてみた
        hit = {"単勝":0,"複勝":0,"馬連":0,"三連複":0,"馬単":0,"三連単":0,"ワイド":0}
        get = 0
        get_umaren = 0
        from_ = 0
        y_test = y_test.values
        odds_test = odds_test.values
        odds_umaren_test = odds_umaren_test.values
     
        
        for i in q_test:
            pred = model.predict(X_test.iloc[from_:from_+i,:],num_iteration=model.best_iteration)
            result = pd.Series(pred).sort_values(ascending=False)
            pred_rank = result.index.values[0:3]
            pred_rank_set = set(pred_rank)
            y_ = pd.Series(y_test[from_:from_+i]).sort_values(ascending=False)
            true_rank = y_.index.values[0:3]
            true_rank_set = set(true_rank)
            odds = pd.Series(odds_test[from_:from_+i])
            odds_umaren = pd.Series(odds_umaren_test[from_:from_+i])
            print(pred_rank,true_rank)
            #単勝
            if true_rank[0] in pred_rank:
                hit["単勝"] += 1/3 #3
                get += odds[true_rank[0]]
            else:
                pass
            
            #複勝、ワイド、三連複、馬単、三連単
            and_set = pred_rank_set & true_rank_set
            if len(and_set) == 3:
                hit["複勝"] += 3/3 #3
                hit["ワイド"] += 3/3 #3
                
                hit["三連複"] += 1/1 #1
                
                hit["三連単"] += 1/6 #6
                
            elif len(and_set) == 2:
                hit["複勝"] += 2/3
                hit["ワイド"] += 1/3
               
                
            elif len(and_set) == 1:
                hit["複勝"] += 1/3
            
            else:
                pass
            
            pred_rank_set = set(pred_rank[0:3])
            true_rank_set = set(true_rank[0:2])
            and_set = pred_rank_set & true_rank_set
            if len(and_set) == 2:
                get_umaren += odds_umaren[true_rank[0]]
                hit["馬連"] += 1/3
                hit["馬単"] += 1/6
            else:
                pass
            
            hit = pd.Series(hit)
           
                
                
            from_ += i
        
        print(hit/len(q_test))
        print("gain:",get/3/len(q_test))
        print("gain_umaren:",get_umaren/300/len(q_test))
        importance = pd.DataFrame(model.feature_importance(), index=X_train.columns.values, columns=['importance'])
        return importance


In [159]:
df = pd.DataFrame()
for i in [2019,2020]:
    df_ = pd.read_csv("horses_data_"+str(i)+".csv")
    df = pd.concat([df,df_],axis=0)
training_umaren(df,1)

[1]	train's ndcg@3: 0.455087	test's ndcg@3: 0.387736
Training until validation scores don't improve for 20 rounds
[2]	train's ndcg@3: 0.542354	test's ndcg@3: 0.369061
[3]	train's ndcg@3: 0.586028	test's ndcg@3: 0.388302
[4]	train's ndcg@3: 0.610362	test's ndcg@3: 0.396793
[5]	train's ndcg@3: 0.630604	test's ndcg@3: 0.401104
[6]	train's ndcg@3: 0.653647	test's ndcg@3: 0.396786
[7]	train's ndcg@3: 0.668279	test's ndcg@3: 0.385139
[8]	train's ndcg@3: 0.678329	test's ndcg@3: 0.385449
[9]	train's ndcg@3: 0.690776	test's ndcg@3: 0.386203
[10]	train's ndcg@3: 0.699348	test's ndcg@3: 0.387937
[11]	train's ndcg@3: 0.709005	test's ndcg@3: 0.393379
[12]	train's ndcg@3: 0.717295	test's ndcg@3: 0.401317
[13]	train's ndcg@3: 0.725213	test's ndcg@3: 0.384548
[14]	train's ndcg@3: 0.735007	test's ndcg@3: 0.388017
[15]	train's ndcg@3: 0.742597	test's ndcg@3: 0.387809
[16]	train's ndcg@3: 0.748563	test's ndcg@3: 0.386537
[17]	train's ndcg@3: 0.754832	test's ndcg@3: 0.391671
[18]	train's ndcg@3: 0.759477	

[11 13 10] [ 1  4 12]
[6 8 7] [10  8  2]
[8 2 1] [1 2 7]
[7 5 4] [4 7 0]
[3 4 9] [6 7 5]
[8 3 5] [9 3 6]
[7 4 1] [2 7 9]
[4 5 9] [16 14  0]
[ 4  0 13] [5 7 0]
[3 1 0] [ 3  7 10]
[ 4 11  1] [ 5 10  6]
[12  5 11] [12  9  7]
[6 5 9] [5 7 6]
[14  3 13] [4 0 9]
[14  3  4] [4 2 3]
[7 5 8] [ 6 11 13]
[ 3 11  6] [5 1 3]
[6 3 1] [6 1 0]
[11  3  1] [14  2  6]
[0 3 5] [2 5 6]
[15  1 12] [15  3  8]
[ 3 13  0] [8 3 9]
[13  1 15] [ 8 10 11]
[ 7  2 13] [2 6 3]
[ 8 11  0] [ 0 10  7]
[10 11  2] [10  2  3]
[5 3 0] [1 0 7]
[ 9  5 13] [13  5  7]
[ 4  8 11] [ 4 11  7]
[10  4  5] [10  5  8]
[ 6 10  8] [1 9 8]
[ 6  0 13] [14  4  2]
[15  4  0] [ 9 12  4]
[ 4  1 13] [ 7 10 14]
[11 14 13] [11  5 12]
[ 5 11 13] [ 7  5 11]
[12  0  4] [12 10  5]
[ 5 13 14] [ 4 10  2]
[12  3  7] [ 3 14 11]
[7 4 1] [7 5 1]
[7 5 1] [ 9 10  3]
[ 5 14  0] [ 5 14  7]
[10  4  2] [6 8 2]
[ 5  0 11] [11 12  9]
[8 4 7] [4 8 5]
[ 0 12 10] [ 7  1 14]
[12  7  9] [12  5  1]
[9 4 1] [ 9 13  6]
[12 14 15] [ 9 12  8]
[13 15  3] [ 7  3 13]
[2 7 5] 

,importance
horse_num,212
horse_name,0
age,468
jockey,260
g_d,178
length,766
weather,350
baba,399
prize,775
studium,48


In [5]:

def prediction(le,model,race_id):
    race_id = "202006020211"
    driver = Chrome(options=options)
    target_url = 'https://race.netkeiba.com/race/shutuba.html?race_id='+race_id+'&rf=race_list'    
    driver.get(target_url)
    html = driver.page_source.encode('euc-jp',"ignore")  
    soup = BeautifulSoup(html, "html.parser")
    df = pd.read_html(target_url)[0]
    #15:45発走 / 芝1600m (左 外)\n/ 天候:晴\n/ 馬場:良
    data_1 = soup.select_one("#page > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData01").text
    result_1 = re.findall(r'/\s([^\s]+)',data_1)
    siba_da = result_1[0][0]
    length = float(result_1[0][1:-1])
    weather = result_1[1][-1]
    baba = result_1[2][-1]
    # 本賞金:3900,1600,980,590,390万円
    data_2 = soup.select_one("#page > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData02 > span:nth-child(10)").text
    result_2 = re.findall(r'本賞金:([^,]+),',data_2)
    prize = float(result_2[0])


    df.drop(columns="お気に入り馬",inplace=True)
    df_new = pd.DataFrame()
    df_new.index = df["馬番"]
    df_new["馬名"] = df["馬名"]
    df_new["性齢"] = df["性齢"]
    df_new["騎手"] = df["騎手"]
    df_new["芝/ダ"] = siba_da
    df_new["距離"] = length
    df_new["天気"] = weather
    df_new["馬場"] = baba
    df_new["賞金"] = prize
    df_new["studium"] = race_id[5]
    df_new["round"] = race_id[-2:]
    df_new["year"] = race_id[:4]


    df = df_new
    df["馬齢"] = [x[1] for x in df["性齢"]]
    df.drop(columns="性齢")

    for i in ["馬名","騎手","芝/ダ","天気","馬場","studium","round","year"]:
          df[i] = le[i].transform(df[i])

    df.rename(columns={
            '馬名':'horse_name',
            '馬齢':'age',
            "騎手":"jockey",
            "芝/ダ":"g_d",
            "距離":"length",
            "天気":"weather",
            "馬場":"baba",
            "賞金":"prize"
    }, inplace=True)
    print(df)
    pred = model.predict(df,num_iteration=model.best_iteration)
    result = pd.DataFrame()
    result["予測結果"] = pred
    result.index = result.index + 1
    result = result.sort_values("予測結果")
    print(result)
    


In [7]:
df = pd.DataFrame()
for i in [2019,2020]:
    df_ = pd.read_csv("horses_data_"+str(i)+".csv")
    df = pd.concat([df,df_],axis=0)
model,le = training_umaren(df)
prediction(le,model,"201705030111")


['horse_num' 'horse_name' 'jockey' 'g_d' 'weather' 'baba' 'studium'
 'round' 'year']
[1]	train's ndcg@3: 0.520841	test's ndcg@3: 0.350888
Training until validation scores don't improve for 20 rounds
[2]	train's ndcg@3: 0.660987	test's ndcg@3: 0.351647
[3]	train's ndcg@3: 0.730121	test's ndcg@3: 0.363208
[4]	train's ndcg@3: 0.766312	test's ndcg@3: 0.362221
[5]	train's ndcg@3: 0.79191	test's ndcg@3: 0.368413
[6]	train's ndcg@3: 0.814299	test's ndcg@3: 0.373451
[7]	train's ndcg@3: 0.825172	test's ndcg@3: 0.374094
[8]	train's ndcg@3: 0.837428	test's ndcg@3: 0.36872
[9]	train's ndcg@3: 0.847284	test's ndcg@3: 0.379592
[10]	train's ndcg@3: 0.855541	test's ndcg@3: 0.386802
[11]	train's ndcg@3: 0.864633	test's ndcg@3: 0.382815
[12]	train's ndcg@3: 0.875469	test's ndcg@3: 0.385105
[13]	train's ndcg@3: 0.882703	test's ndcg@3: 0.382788
[14]	train's ndcg@3: 0.890002	test's ndcg@3: 0.379692
[15]	train's ndcg@3: 0.896833	test's ndcg@3: 0.380907
[16]	train's ndcg@3: 0.901674	test's ndcg@3: 0.386934
[

ValueError: Index data must be 1-dimensional